In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epoch = 100

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [24]:
FGSM = onp.load('./npy/cifar-targeted-fgsm-x.npy')
I_FGSM = onp.load('./npy/cifar-targeted-ifgsm-x.npy')

In [25]:
y_test = onp.load("./npy/cifar-targeted-clean-y.npy")

In [26]:
x_test = onp.load("./npy/cifar-targeted-clean-x.npy")

In [27]:
layers = tf.keras.layers

In [28]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
out = layers.Dense(10)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [29]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-3
    elif epoch < 55:
        return 1e-2
    elif epoch < 75:
        return 1e-3
    else:
        return 1e-4

In [30]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [31]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [32]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16777472  
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [33]:
x_train_all, x_test_all = x_train_all.reshape(-1, 32, 32, 3), x_test_all.reshape(-1, 32, 32, 3)

In [34]:
model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=200, validation_split=0.1, callbacks=[callback])

Epoch 1/200
176/176 [==============================] - 5s 29ms/step - loss: 2.2892 - accuracy: 0.1126 - val_loss: 2.2722 - val_accuracy: 0.1268
Epoch 2/200
176/176 [==============================] - 2s 14ms/step - loss: 2.2524 - accuracy: 0.1522 - val_loss: 2.2256 - val_accuracy: 0.1964
Epoch 3/200
176/176 [==============================] - 4s 22ms/step - loss: 2.1951 - accuracy: 0.2136 - val_loss: 2.1562 - val_accuracy: 0.2260
Epoch 4/200
176/176 [==============================] - 2s 13ms/step - loss: 2.1282 - accuracy: 0.2441 - val_loss: 2.1002 - val_accuracy: 0.2564
Epoch 5/200
176/176 [==============================] - 2s 13ms/step - loss: 2.0755 - accuracy: 0.2676 - val_loss: 2.0499 - val_accuracy: 0.2742
Epoch 6/200
176/176 [==============================] - 2s 13ms/step - loss: 2.0020 - accuracy: 0.2908 - val_loss: 1.8983 - val_accuracy: 0.3356
Epoch 7/200
176/176 [==============================] - 2s 13ms/step - loss: 1.7919 - accuracy: 0.3704 - val_loss: 1.7417 - val_accuracy:

176/176 [==============================] - 2s 13ms/step - loss: 0.0244 - accuracy: 0.9985 - val_loss: 1.8843 - val_accuracy: 0.6292
Epoch 114/200
176/176 [==============================] - 2s 13ms/step - loss: 0.0243 - accuracy: 0.9985 - val_loss: 1.8850 - val_accuracy: 0.6292
Epoch 115/200
176/176 [==============================] - 2s 13ms/step - loss: 0.0243 - accuracy: 0.9985 - val_loss: 1.8857 - val_accuracy: 0.6292
Epoch 116/200
176/176 [==============================] - 2s 13ms/step - loss: 0.0242 - accuracy: 0.9985 - val_loss: 1.8862 - val_accuracy: 0.6288
Epoch 117/200
176/176 [==============================] - 2s 13ms/step - loss: 0.0242 - accuracy: 0.9985 - val_loss: 1.8871 - val_accuracy: 0.6292
Epoch 118/200
176/176 [==============================] - 2s 13ms/step - loss: 0.0241 - accuracy: 0.9986 - val_loss: 1.8877 - val_accuracy: 0.6290
Epoch 119/200
176/176 [==============================] - 2s 13ms/step - loss: 0.0241 - accuracy: 0.9985 - val_loss: 1.8882 - val_accuracy:

In [19]:
model.load_weights('./model_weights/simple_cnn.h5')

In [20]:
# model.load_weights('./simple_cnn_mse.h5')

In [21]:
ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [22]:
@tf.function
def targeted_fgsm(x, y_target, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_target, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x - eps * tf.sign(grad), 0, 1)

@tf.function
def untargeted_fgsm(x, y_true, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_true, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x + eps * tf.sign(grad), 0, 1)

In [ ]:
targeted_fgsm(x, y_target, model, eps)

In [32]:
# tmp = untargeted_fgsm(tf.convert_to_tensor(clean_x), clean_y, model, 0.03)

In [33]:
clean_x = onp.load('./cifar-clean-x.npy')
clean_y = onp.load('./cifar-clean-y.npy')

In [34]:
tmp = onp.load('./cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')

In [42]:
model.evaluate(clean_x, clean_y)

4/4 [==============================] - 0s 2ms/step - loss: 1.2249 - accuracy: 0.6719


[1.2249499559402466, 0.671875]

In [43]:
model.evaluate(tmp, clean_y)

4/4 [==============================] - 0s 2ms/step - loss: 2.6337 - accuracy: 0.3359


[2.6336843967437744, 0.3359375]

In [20]:
tmp = targeted_fgsm(tf.convert_to_tensor(x_test), onp.tile(onp.eye(10)[0], 4096).reshape(-1, 10), model, 0.03)

In [21]:
itmp = targeted_ifgsm(tf.convert_to_tensor(x_test), onp.tile(onp.eye(10)[0], 4096).reshape(-1, 10), model, 0.0003*1.1, 100)

In [41]:
sum(onp.argmax(model(itmp), axis=1)[onp.argmax(model(x_test), axis=1) == onp.argmax(y_test, axis=1)]==0) /\
sum(onp.argmax(model(x_test), axis=1) == onp.argmax(y_test, axis=1))

0.9648183556405354

In [35]:
onp.save('cifar-tageted-simple-cnn-ifgsm.npy' ,itmp)

NameError: name 'itmp' is not defined